<a href="https://colab.research.google.com/github/Pravardhan-21/historical_data_analysis/blob/main/Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Libraries and paths

In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import timedelta
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix
import statsmodels.api as sm
HIST_PATH="/content/historical_data.csv"
FG_PATH="/content/fear_greed_index.csv"
OUTPUT_DIR="/content/output"
MAX_LAG_DAYS=5
ROLLING_WINDOW=30
os.makedirs(OUTPUT_DIR,exist_ok=True)


#Utility functions

In [ ]:
def find_first(cols, candidates):
    for c in candidates:
        if c in cols:
            return c
    return None
def safe_read_csv(path):
    return pd.read_csv(path)

#Load Data and Detect Columns

In [ ]:
#Load both files
hist=safe_read_csv(HIST_PATH)
fg=safe_read_csv(FG_PATH)
#Clean column names
hist.columns=[c.strip() for c in hist.columns]
fg.columns=[c.strip() for c in fg.columns]
#Identify columns
date_col=find_first(hist.columns,['Timestamp IST','Timestamp','Date','date'])
trader_col=find_first(hist.columns,['Account','trader_id','account'])
pnl_col= find_first(hist.columns,['Closed PnL','PnL','pnl'])
size_col=find_first(hist.columns,['Size USD','size_usd','notional'])
price_cols=[c for c in hist.columns if 'price' in c.lower()]
print("Detected -> date:",date_col,"| trader:",trader_col," pnl:",pnl_col," size:",size_col)


Detected -> date: Timestamp IST | trader: Account  pnl: Closed PnL  size: Size USD


#Compute Trade Returns

In [ ]:
# Parse dates
hist['date']=pd.to_datetime(hist[date_col],errors='coerce').dt.normalize()
# Compute per-trade return
if pnl_col and size_col:
    hist['trade_return']=hist[pnl_col]/hist[size_col].replace({0: np.nan})
elif pnl_col:
    hist['trade_return']=hist[pnl_col]
elif len(price_cols)>=2:
    hist['trade_return']=(hist[price_cols[1]]-hist[price_cols[0]])/hist[price_cols[0]]
else:
    raise RuntimeError("No pnl/size or price columns available")
hist=hist.dropna(subset=['date']).reset_index(drop=True)

#Per-Trader Summary

In [ ]:
if trader_col:
    grouped=hist.groupby(trader_col)
    per_trader=pd.DataFrame({
        trader_col:grouped.size().index,
        'trades':grouped.size().values,
        'total_pnl':grouped[pnl_col].sum() if pnl_col else np.nan,
        'avg_return':grouped['trade_return'].mean().values,
        'median_return':grouped['trade_return'].median().values,
        'std_return':grouped['trade_return'].std().values,
        'win_rate':grouped['trade_return'].apply(lambda s: (s>0).mean()).values
    })
    per_trader['sharpe_like']=per_trader['avg_return']/per_trader['std_return'].replace({0: np.nan})
    per_trader.to_csv(os.path.join(OUTPUT_DIR,"per_trader_summary.csv"),index=False)
    print("per_trader_summary.csv saved")


per_trader_summary.csv saved


#Prepare Sentiment Data

In [ ]:
fg_date_col=find_first(fg.columns,['date','Date','Timestamp'])
fg['date']=pd.to_datetime(fg[fg_date_col],errors='coerce').dt.normalize()
#pick sentiment column
numeric_cols=fg.select_dtypes(include=[np.number]).columns.tolist()
sentiment_col=numeric_cols[0] if len(numeric_cols)>0 else fg.columns[1]
fg2=fg[['date',sentiment_col]].rename(columns={sentiment_col:'sentiment'}).dropna().drop_duplicates()
full_dates=pd.DataFrame({'date':pd.date_range(fg2['date'].min(),fg2['date'].max(),freq='D')})
fg_daily=full_dates.merge(fg2,on='date',how='left').sort_values('date')
fg_daily['sentiment']=fg_daily['sentiment'].ffill().bfill()

#Daily Aggregates and Merge

In [ ]:
daily=hist.groupby('date').agg(
    daily_ret_mean=('trade_return','mean'),
    daily_ret_sum=('trade_return','sum'),
    trades=('trade_return','count')
).reset_index()
merged=pd.merge(daily,fg_daily,on='date',how='left').sort_values('date')
merged['sentiment']=merged['sentiment'].ffill().bfill()
merged.to_csv(os.path.join(OUTPUT_DIR,"merged_daily_sentiment_returns.csv"),index=False)
print("merged_daily_sentiment_returns.csv saved")


merged_daily_sentiment_returns.csv saved


#Lagged and Rolling Correlations

In [ ]:
lag_results=[]
for lag in range(0, MAX_LAG_DAYS+1):
    merged[f'sent_lag_{lag}']=merged['sentiment'].shift(lag)
    valid=merged.dropna(subset=[f'sent_lag_{lag}','daily_ret_mean'])
    corr=valid['daily_ret_mean'].corr(valid[f'sent_lag_{lag}'])
    lag_results.append({'lag_days':lag,'corr':corr})
lag_df=pd.DataFrame(lag_results)
lag_df.to_csv(os.path.join(OUTPUT_DIR,"lagged_correlations.csv"),index=False)
# rolling correlation plot
merged['rolling_corr']=merged['daily_ret_mean'].rolling(window=ROLLING_WINDOW,min_periods=10).corr(merged['sentiment'])
plt.plot(merged['date'],merged['rolling_corr'])
plt.title(f"{ROLLING_WINDOW}-day rolling correlation")
plt.savefig(os.path.join(OUTPUT_DIR,"rolling_corr.png"));
plt.close()

#Per-Trader Regressions

In [ ]:
if trader_col:
    daily_trader=hist.groupby([trader_col,'date']).agg(daily_ret=('trade_return','mean')).reset_index()
    dt=pd.merge(daily_trader,fg_daily, on='date',how='left').sort_values('date')
    dt['sentiment']=dt['sentiment'].ffill().bfill()
    results=[]
    for trader,grp in dt.groupby(trader_col):
        grp=grp.copy()
        for lag in range(MAX_LAG_DAYS+1):
            grp[f'sent_lag_{lag}']=grp['sentiment'].shift(lag)
        grp=grp.dropna()
        if len(grp)<10:continue
        X=sm.add_constant(grp[[f'sent_lag_{l}'for l in range(MAX_LAG_DAYS+1)]])
        y=grp['daily_ret']
        mod=sm.OLS(y,X).fit()
        results.append({
            'trader':trader,
            'coef_sent_0':mod.params.get('sent_lag_0',np.nan),
            'pval_sent_0':mod.pvalues.get('sent_lag_0',np.nan),
            'r2':mod.rsquared,
            'n_obs':len(grp)
        })
    pd.DataFrame(results).to_csv(os.path.join(OUTPUT_DIR,"per_trader_sentiment_regs.csv"),index=False)
    print("per_trader_sentiment_regs.csv saved")


per_trader_sentiment_regs.csv saved


#Event Study on Extreme Sentiment

In [ ]:
p_top=merged['sentiment'].quantile(0.9)
p_bot=merged['sentiment'].quantile(0.1)
extreme_days=merged[(merged['sentiment']>=p_top)|(merged['sentiment']<=p_bot)]
window=3
frames=[]
for _,row in extreme_days.iterrows():
    day=row['date']
    rng=pd.date_range(day-pd.Timedelta(days=window),day+pd.Timedelta(days=window))
    sub=merged[merged['date'].isin(rng)][['date','daily_ret_mean']].copy()
    sub['day_offset']=(sub['date']-day).dt.days
    sub['event_date']=day
    sub['type']='top' if row['sentiment']>=p_top else 'bot'
    frames.append(sub)
events=pd.concat(frames)
summary=events.groupby(['type','day_offset']).daily_ret_mean.mean().reset_index()
summary.to_csv(os.path.join(OUTPUT_DIR,"event_study_extreme_sentiment.csv"),index=False)

#Clustering Traders

In [ ]:
if trader_col:
    feat=per_trader[['trades','avg_return','win_rate','sharpe_like']].dropna()
    X=StandardScaler().fit_transform(feat)
    kmeans=KMeans(n_clusters=3,random_state=42).fit(X)
    per_trader.loc[feat.index,'cluster']=kmeans.labels_
    per_trader.to_csv(os.path.join(OUTPUT_DIR,"per_trader_clusters.csv"),index=False)
    print("per_trader_clusters.csv saved")

per_trader_clusters.csv saved


#Predictive Model

In [ ]:
df=hist.copy()
df['win']=(df['trade_return']>0).astype(int)
df=pd.merge(df,fg_daily,on='date',how='left')
df['sent_lag_1']=df['sentiment'].shift(1).fillna(df['sentiment'])
X=df[['sentiment','sent_lag_1']].fillna(0)
y=df['win']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)
clf=LogisticRegression(max_iter=1000).fit(X_train,y_train)
y_pred=clf.predict(X_test);
y_prob=clf.predict_proba(X_test)[:,1]
print("Accuracy:",accuracy_score(y_test,y_pred))
print("AUC:",roc_auc_score(y_test,y_prob))

Accuracy: 0.5776585673714105
AUC: 0.5050363355401556
